## Importar datos

In [214]:
import pandas as pd

df = pd.read_csv("BI.csv")
# Eliminar cliente inusual
df = df[df["ClientID"] != "C442030"]
# Transforma a formato de fecha
df["DATE"] = pd.to_datetime(df["DATE"], format="%d/%m/%Y")
df

,DATE,ClientID,MOVEMENT,BALANCE_IN,TRANSFEROR,BALANCE_OUT,RECEPIENT,AGENT ID,STATUS,CHANNEL,aux,IN,OUT,NUM_MOVEMENTS
0,2017-09-30,C689751,in,1073.05,Competitor 4,0.00,Competitor 9,A6022,Inactive,CHANNEL 5,1,1,0,2
1,2018-12-31,C689751,out,0.00,Competitor 4,1066.17,Competitor 7,A6022,Inactive,CHANNEL 5,1,0,1,2
2,2019-08-31,C689750,in,667984.98,Competitor 12,0.00,Competitor 9,A17817,Active,CHANNEL 4,1,1,0,1
3,2017-07-31,C689749,in,15180.39,Competitor 14,0.00,Competitor 9,A24328,Active,CHANNEL 4,1,1,0,1
4,2020-03-31,C689748,in,142252.63,Competitor 7,0.00,Competitor 12,A12016,Active,CHANNEL 1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749197,2017-03-31,C5,out,0.00,Competitor 8,294338.39,Competitor 14,A30677,Inactive,CHANNEL 5,1,0,1,1
749198,2019-08-31,C4,in,19651.88,Competitor 2,0.00,Competitor 2,A14667,Active,CHANNEL 1,1,1,0,1
749199,2017-01-31,C3,out,0.00,Competitor 7,175529.62,Competitor 2,A18024,Inactive,CHANNEL 5,1,0,1,1
749200,2018-02-28,C2,out,0.00,Competitor 3,0.00,Competitor 2,A13087,Inactive,CHANNEL 5,1,0,1,1


## Filtrado de clientes

In [215]:
clients = df.groupby(by=["ClientID"]).agg({"aux": "sum", "AGENT ID": "count"})
# Filtrar clientes con aux > 2
clients = clients[clients["aux"] > 2]
# Guarda todos los clientes de interes en una lista
list_clients = clients.index
clients

,aux,AGENT ID
ClientID,,
C1008,3,3
C10104,3,3
C101167,4,4
C101355,3,3
C101396,3,3
...,...,...
C97979,3,3
C98253,3,3
C98418,3,3


## Dataframe solo con los clientes de interés

In [216]:
# Nuevo dataframe con los clientes filtrados
new_df = df.set_index("ClientID").loc[list_clients]
new_df = new_df.reset_index()
new_df

,ClientID,DATE,MOVEMENT,BALANCE_IN,TRANSFEROR,BALANCE_OUT,RECEPIENT,AGENT ID,STATUS,CHANNEL,aux,IN,OUT,NUM_MOVEMENTS
0,C1008,2016-01-31,out,0.00,Competitor 7,160982.99,Competitor 14,A4325,Inactive,CHANNEL 4,1,0,1,3
1,C1008,2018-03-31,in,191914.97,Competitor 14,0.00,Competitor 14,A11348,Active,CHANNEL 1,1,1,0,3
2,C1008,2019-10-31,out,0.00,Competitor 14,222071.22,Competitor 7,A11348,Active,CHANNEL 1,1,0,1,3
3,C10104,2017-05-31,in,532581.83,Competitor 14,0.00,Competitor 12,A23944,Active,CHANNEL 1,1,1,0,3
4,C10104,2019-01-31,out,0.00,Competitor 14,601029.62,Competitor 7,A23944,Active,CHANNEL 1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4962,C98816,2017-06-30,in,117949.13,Competitor 13,0.00,Competitor 13,A23623,Active,CHANNEL 1,1,1,0,3
4963,C98816,2020-02-29,out,0.00,Competitor 13,125110.90,Competitor 6,A23623,Active,CHANNEL 1,1,0,1,3
4964,C99699,2016-04-30,in,775560.77,Competitor 6,0.00,Competitor 6,A21626,Inactive,CHANNEL 1,1,1,0,3
4965,C99699,2018-01-31,out,0.00,Competitor 6,905897.83,Competitor 6,A21626,Inactive,CHANNEL 5,1,0,1,3


## Lista de agentes de interés

In [220]:
# Lista de agentes contenidas en el nuevo dataframe
agents = new_df["AGENT ID"].unique()
print("Agentes totales:",len(agents))
print(agents)

Agentes totales: 1379
['A4325' 'A11348' 'A23944' ... 'A13968' 'A22473' 'A8137']


## Iteración para todos los clientes

### Método 1 (Combinaciones)

In [206]:
from itertools import combinations

table = pd.DataFrame(columns=["AGENTS"])

# Itera sobre todos los clientes
for client in list_clients:
    # Filtra dataframe por cliente
    client_df = new_df[new_df["ClientID"] == client]
    # Enlista todos los agentes que comparten un cliente
    agent_per_client = client_df["AGENT ID"].sort_values().unique()
    # Crea dataframe con todas las combinaciones de los agentes
    agent_combinations = pd.DataFrame({"AGENTS":list(combinations(agent_per_client,2))})
    # Añade la columna cliente con el cliente que comparten estas combinaciones
    agent_combinations["CLIENT"] = client
    # Concatena los dataframes generados
    table = pd.concat([table,agent_combinations])

# Agrupa la tabla por combinaciones de clientes
table = table.groupby("AGENTS").count()
# Convierte la combinación de agente en 2 listas
two_lists = list(zip(*table.index))
table["AGENT1"] = list(two_lists[0])
table["AGENT2"] = list(two_lists[1])
# Elimina la columna AGENTS que contiene la combinación de agentes
table.reset_index(inplace=True)
table.drop("AGENTS", axis=1, inplace=True)
# Reconfigura la tabla
table = table[["AGENT1", "AGENT2", "CLIENT"]]
table = table.sort_values("CLIENT", ascending=False)
table.head(10)

,AGENT1,AGENT2,CLIENT
486,A16558,A25272,14
592,A17700,A23944,9
942,A21662,A6098,9
596,A17700,A30621,8
466,A16483,A21856,8
1267,A30621,A8410,8
1292,A8409,A8464,7
385,A15169,A25957,7
951,A21856,A7004,7
936,A21626,A7004,6


In [ ]:
table.to_csv("tabla_red_social.csv")

### Método 2 (Iteradores)

#### Función intersección de clientes

In [ ]:
def intersection(lst1, lst2):
    """Retorna una lista de los clientes comunes entre dos agentes.
    Argumentos de entrada
    lst1: ingresa la lista de clientes del agente1
    lst2: ingresa la lista de clientes del agente2
    """
    lst1 = set(lst1)
    lst2 = set(lst2)
    return list(lst1.intersection(lst2))

In [ ]:
# Prueba de la función
print("Intersecciones de ({}, {})".format(agents[464],agents[321]))
common_client = intersection(new_df[new_df["AGENT ID"] == agents[464]]["ClientID"],
             new_df[new_df["AGENT ID"] == agents[321]]["ClientID"])
print(common_client)

Intersecciones de (A21626, A7004)
['C4862', 'C81868', 'C350422', 'C656224', 'C230018', 'C99699']


#### Proceso de iteración

In [104]:
table_list = []
x = 0
n = len(agents)

# Itera sobre todas las combinaciones de agentes
for i in range(n):
    for j in range(i+1,n):
        # Encuentra los clientes en común
        common_client = intersection(new_df[new_df["AGENT ID"] == agents[i]]["ClientID"],
                                     new_df[new_df["AGENT ID"] == agents[j]]["ClientID"])
        # Cuenta los clientes en común
        x = len(common_client)
        # Agrega a la tabla si hay al menos un cliente en comun
        if x > 0:
            table_list.append([agents[i],agents[j],x])
    # Imprime cada 10 agentes el estatus del progreso
    if i % 10 == 0:
        print("\rAgentes analizados: {}/{}".format(i,n),end="")

# Imprime y organiza la tabla
table = pd.DataFrame(table_list, columns=["AGENTID1", "AGENTID2", "CLIENTS"])
table.sort_values("CLIENTS", ascending=False)

Agentes analizados: 1370/1379
Finalizado en 971.51 min


In [107]:
table.to_csv("tabla_red_social.csv")

## Social Network

In [1]:
import pandas as pd
table = pd.read_csv("tabla_red_social.csv")
table.drop(["Unnamed: 0"], axis=1, inplace=True)
table.sort_values("CLIENTS", ascending=False)

,AGENTID1,AGENTID2,CLIENTS
486,A16558,A25272,14
592,A17700,A23944,9
942,A21662,A6098,9
596,A17700,A30621,8
466,A16483,A21856,8
...,...,...,...
441,A15818,A23867,1
438,A15805,A19046,1
437,A15772,A23911,1
436,A15772,A20993,1


In [2]:
import networkx as nx

G = nx.from_pandas_edgelist(table,
                            source="AGENTID1",
                            target="AGENTID2",
                            edge_attr="CLIENTS")

In [3]:
from pyvis.network import Network

nt = Network(height="800px",width="800px",notebook=True)
nt.from_nx(G)
nt.show("social_net.html")